# Lab 8 - Integer Programming - BnB for MIP

Information on group members:

1) 148279, Lukasz Kosturski <br>
2) 148257, Daniel Jankowski

In [66]:
from pulp import *  
import numpy as np
import pandas as pd

1) Given is the below MIP problem. Note that the first 5 variables are of an integer type with specified upper bounds

In [67]:
def getProblem(relaxed = False, return_init_matrices=False):
    
    A = [
        [0,3,2,0,0,0,-3,-1,0,0],
        [1,1,0,2,0,0,0,-1,2,1],
        [0,0,2,-2,3,0,-2,2,1,0],
        [0,0,2,0,0,-1,0,0,0,1],
        [0,2,0,0,0,-2,0,0,0,1],
        [1,4,0,0,0,0,-3,6,2,0],
        [2,2,0,0,2,2,0,0,2,2],
        [0,0,3,0,-1,1,0,-1,0,1],
        [0,0,0,0,5,0,1,1,0,3],
        [2,-7,0,0,0,1,0,8,2,0]]
    b = [10,15,20,20,30,50,40,20,25,25]
    c = [5, 7, 5, 5, 5, 5, 7, 4, 9, 10]
    uB = [5, 8, 4, 5, 4, 5, 5, 3, 3, 3]
    
    problem = LpProblem(name="bnb-problem", sense=LpMaximize)
    
    ### 5 integers and 3 continuous (if relaxed, 8 cont.)
    cat = ['Integer' for i in range(5)] + ['Continuous' for i in range(5)]
    if relaxed: cat = ['Continuous' for i in range(5)] + ['Continuous' for i in range(5)]
        
    x = [LpVariable(name="x"+ str(i+1), lowBound=0, upBound=uB[i], cat = cat[i]) for i in range(10)]
    
    for r in range(10):
        expr = lpSum([x[j] * A[r][j] for j in range(10)])
        problem += LpConstraint(e=expr, sense = -1, name = "baseC"+str(r+1), rhs = b[r])
        
    obj_func = lpSum([x[j] * c[j] for j in range(10)])
    problem += obj_func

    if return_init_matrices:
        return x, problem, {'A': A, 'b':b, 'c':c, 'uB':uB}
    
    return x, problem

x, P = getProblem()
print(P)

bnb-problem:
MAXIMIZE
5*x1 + 10*x10 + 7*x2 + 5*x3 + 5*x4 + 5*x5 + 5*x6 + 7*x7 + 4*x8 + 9*x9 + 0
SUBJECT TO
baseC1: 3 x2 + 2 x3 - 3 x7 - x8 <= 10

baseC2: x1 + x10 + x2 + 2 x4 - x8 + 2 x9 <= 15

baseC3: 2 x3 - 2 x4 + 3 x5 - 2 x7 + 2 x8 + x9 <= 20

baseC4: x10 + 2 x3 - x6 <= 20

baseC5: x10 + 2 x2 - 2 x6 <= 30

baseC6: x1 + 4 x2 - 3 x7 + 6 x8 + 2 x9 <= 50

baseC7: 2 x1 + 2 x10 + 2 x2 + 2 x5 + 2 x6 + 2 x9 <= 40

baseC8: x10 + 3 x3 - x5 + x6 - x8 <= 20

baseC9: 3 x10 + 5 x5 + x7 + x8 <= 25

baseC10: 2 x1 - 7 x2 + x6 + 8 x8 + 2 x9 <= 25

VARIABLES
0 <= x1 <= 5 Integer
x10 <= 3 Continuous
0 <= x2 <= 8 Integer
0 <= x3 <= 4 Integer
0 <= x4 <= 5 Integer
0 <= x5 <= 4 Integer
x6 <= 5 Continuous
x7 <= 5 Continuous
x8 <= 3 Continuous
x9 <= 3 Continuous



In [68]:
solution = P.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: May  6 2022 

command line - cbc /tmp/50ff67307a8a4d09ac77a75db8ac32ea-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/50ff67307a8a4d09ac77a75db8ac32ea-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 83 RHS
At line 94 BOUNDS
At line 105 ENDATA
Problem MODEL has 10 rows, 10 columns and 47 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 211.333 - 0.00 seconds
Cgl0004I processed model has 7 rows, 10 columns (5 integer (0 of which binary)) and 36 elements
Cbc0012I Integer solution of -206 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0012I Integer solution of -207 found by DiveCoefficient after 166 iterations and 0 nodes (0.06 seconds)
Cbc0031I 5 added rows had average density of 7.6
Cbc0013I At root node, 5 cuts changed objective from -211.33333 to -207

2) The below function returns None if the problem has no feasible solutions. Otherwise, it returns a tuple: objective function values and a vector of decision variables. 

In [69]:
def getSolution(x, problem):
    status = problem.solve()
    if problem.status != 1: return None
    return problem.objective.value(), [_.value() for _ in x]

3) PuLP can solve MIP problems. Hence, the "relaxed" flag can be set to False. Solve the problem and analyze the obtained outcome.  

In [70]:
x, problem = getProblem(relaxed = False)
# print(getSolution(x, problem))

4) Now, compare this solution with the one obtained for the relaxed LP problem: 

In [71]:
x, problem = getProblem(relaxed = True)
# print(getSolution(x, problem))

5) Your task is to implement the Branch and Bound Algorithm for solving MIP problems. You can use the PuLP library for solving the relaxed LP subproblems. <br>
<ul> 
    <li> Firstly, as a node selection policy, implement the default DFS-like strategy as shown in the lecture (generate both children in one iteration; prioritize the left children, i.e., associated with the "<=" constraint). As for the variable selection policy, take the one with the lowest index  (default, arbitrary selection). 
<li> Identify how many LP relaxed problems have to be solved to find the optimum. Note that such a number was reported to be 35 for the default policies. However, it may vary slightly for different solvers due to possible multiple sub-optima.
<li> Propose at least 3 new node and variable selection policies with the aim of minimizing the number of solver runs required to reach the optimum.  Try getting below 20. 
    </ul>

In [151]:
from math import floor, ceil

def solve(branching_policy='dfs'):

    var_mip, _ = getProblem(relaxed=False) # get MIP problem to save information about integer/continuous variables
    integer_vars_inx = [i for i in range(len(var_mip)) if var_mip[i].cat=="Integer"] # indiced of integer variables

    x, P, matrices = getProblem(relaxed=True, return_init_matrices=True) # get initial relaxed problem to be solved as a first step

    constraints_count = np.count_nonzero(np.array(matrices['A'])[:,integer_vars_inx], axis=0)
    
    integer_vars_priority_inx = [idx for _,idx in sorted(zip(constraints_count, integer_vars_inx),reverse=True)]

    # Create a queue to store the nodes of the branch and bound tree
    nodes = []
    if branching_policy =='equal_split' or branching_policy == 'smaller_range_first':
        ranges = []
    visited_nodes = {}

    #best found objective value and variables values
    best_solution_value = -float('inf')  # the same as LowBound
    best_solution_vars = []

    # low bound
    LB = -float('inf')

    # Add the root node to the queue
    nodes.append(P)

    if branching_policy =='equal_split' or branching_policy == 'smaller_range_first':
        uB = matrices['uB']
        ranges.append([(0,uB[i]) for i in integer_vars_inx])


    # Keep track of the number of LP relaxed problems solved
    subproblems_solved = 0
    infeasible_count = 0
    while nodes:
        # Pop the next node from the queue
        node = nodes.pop()
        if branching_policy == 'equal_split' or branching_policy == 'smaller_range_first':
            curr_ranges = ranges.pop()


        # Check if the LP relaxation has a feasible solution
        solution = getSolution(x, node)
        subproblems_solved += 1
        if solution is not None:
            
            problem_objective, vars_values = solution

            
            # Check if the LP relaxation has an integer feasible solution
            if all(vars_values[inx].is_integer() for inx in integer_vars_inx):
                if problem_objective > best_solution_value:
                    best_solution_value = problem_objective
                    best_solution_vars = vars_values
                
            else:

                # if LP relaxation of current node has worse objective than any known integer solution
                if problem_objective < best_solution_value:
                    # prune this branch
                    continue
                

                # Now - generate two children 
                if branching_policy == 'dfs':
                    # feasible solution but it is not integer
                    var_to_branch_on = None
                    for inx in integer_vars_inx: 
                        if not vars_values[inx].is_integer(): # take first possible variable which should has integer value but doesn't have
                            var_to_branch_on = inx
                            break
                    if var_to_branch_on is not None:
                        # Generate the right child (associated with the ">=" constraint)
                        right_child = node.copy()
                        right_child += x[inx] >= ceil(x[inx].value())
                        nodes.append(right_child)
                        
                        # Generate the left child (associated with the "<=" constraint)
                        left_child = node.copy()
                        left_child += x[inx] <= floor(x[inx].value())
                        nodes.append(left_child)
                        # add left child at the end, because we want to prioritize left one (this one will be poped from the list earlier)

                if branching_policy == 'more_constraints_first':
                    # feasible solution but it is not integer
                    var_to_branch_on = None
                    for inx in integer_vars_priority_inx: 
                        if not vars_values[inx].is_integer(): # take first possible variable which should has integer value but doesn't have
                            var_to_branch_on = inx
                            break
                    if var_to_branch_on is not None:
                        # Generate the right child (associated with the ">=" constraint)
                        right_child = node.copy()
                        right_child += x[inx] >= ceil(x[inx].value())
                        nodes.append(right_child)
                        
                        # Generate the left child (associated with the "<=" constraint)
                        left_child = node.copy()
                        left_child += x[inx] <= floor(x[inx].value())
                        nodes.append(left_child)
                        # add left child at the end, because we want to prioritize left one (this one will be poped from the list earlier)

                    
                
                if branching_policy == 'smaller_range_first':

                    var_to_branch_on = None
                    
                    curr_ranges_lengths = [high - low for (low,high) in curr_ranges]
                    integer_vars_priority_inx = [idx for _,idx in sorted(zip(curr_ranges_lengths, integer_vars_inx),reverse=True)] #TODO experiment


                    for inx in integer_vars_priority_inx: 
                        if not vars_values[inx].is_integer(): # take first possible variable which should has integer value but doesn't have
                            var_to_branch_on = inx
                            break
                    assert var_to_branch_on is not None
                    
                    # get the middle value of the range
                    split_val = floor(x[inx].value())

                    right_child = node.copy()
                    right_child += x[inx] >= split_val+1
                    right_ranges = curr_ranges.copy()
                    right_ranges[inx] = (split_val+1, right_ranges[inx][1])
                    nodes.append(right_child)
                    ranges.append(right_ranges)
                    
                    # Generate the left child (associated with the "<=" constraint)
                    left_child = node.copy()
                    left_child += x[inx] <= split_val
                    left_ranges = curr_ranges.copy()
                    left_ranges[inx] = (left_ranges[inx][0], split_val)
                    nodes.append(left_child)
                    ranges.append(left_ranges)


                if branching_policy == 'equal_split':
                    var_to_branch_on = None
                    high, low = None, None
                    
                    # calculate the lengths of ranges
                    # range_lengths = [high - low for (low, high) in curr_ranges]
                    # get the first variable for which the range_length is not zero
                    for inx in integer_vars_inx:
                        low, high = curr_ranges[inx]
                        if high-low != 0:
                            var_to_branch_on = inx
                            break
                    assert var_to_branch_on is not None
                    
                    # get the middle value of the range
                    mid = floor((high+low)/2)

                    # Generate the left child (associated with the "<=" constraint)
                    left_child = node.copy()
                    left_child += x[inx] <= mid
                    left_ranges = curr_ranges.copy()
                    left_ranges[inx] = (low, mid)
                    nodes.append(left_child)
                    ranges.append(left_ranges)

                    right_child = node.copy()
                    right_child += x[inx] >= mid+1
                    right_ranges = curr_ranges.copy()
                    right_ranges[inx] = (mid+1, high)
                    nodes.append(right_child)
                    ranges.append(right_ranges)

                
        else:
            infeasible_count += 1

    print("Number of LP relaxed problems solved:", subproblems_solved)

    print("Optimal solution value:", best_solution_value)
    print("Variables values: ", best_solution_vars)


solve(branching_policy='more_constraints_first')


Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: May  6 2022 

command line - cbc /tmp/1e64f5266fa9417a9b3c0ce58913f647-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/1e64f5266fa9417a9b3c0ce58913f647-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 73 RHS
At line 84 BOUNDS
At line 95 ENDATA
Problem MODEL has 10 rows, 10 columns and 47 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 7 (-3) rows, 10 (0) columns and 36 (-11) elements
0  Obj -0 Dual inf 64.49999 (10)
6  Obj 211.33333
Optimal - objective value 211.33333
After Postsolve, objective 211.33333, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 211.3333333 - 6 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.01   (Wallclock seconds):       0.00

Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date